<a href="https://colab.research.google.com/github/renweizhukov/jupyter-lab-notebook/blob/main/hugging-face-agents-course/monitoring_and_evaluating_agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bonus Unit 2: Observability and Evaluation of Agents

In this tutorial, we will learn how to **monitor the internal steps (traces) of our AI agent** and **evaluate its performance** using open-source observability tools.

The ability to observe and evaluate an agent’s behavior is essential for:
- Debugging issues when tasks fail or produce suboptimal results
- Monitoring costs and performance in real-time
- Improving reliability and safety through continuous feedback

This notebook is part of the [Hugging Face Agents Course](https://www.hf.co/learn/agents-course/unit1/introduction).

## Exercise Prerequisites 🏗️

Before running this notebook, please be sure you have:

🔲 📚  **Studied** [Introduction to Agents](https://huggingface.co/learn/agents-course/unit1/introduction)

🔲 📚  **Studied** [The smolagents framework](https://huggingface.co/learn/agents-course/unit2/smolagents/introduction)

## Step 0: Install the Required Libraries

We will need a few libraries that allow us to run, monitor, and evaluate our agents:

In [21]:
%pip install 'smolagents[telemetry]'
%pip install opentelemetry-sdk opentelemetry-exporter-otlp openinference-instrumentation-smolagents
%pip install langfuse datasets 'smolagents[gradio]' gradio

## Step 1: Instrument Your Agent

In this notebook, we will use [Langfuse](https://langfuse.com/) as our observability tool, but you can use **any other OpenTelemetry-compatible service**. The code below shows how to set environment variables for Langfuse (or any OTel endpoint) and how to instrument your smolagent.

**Note:** If you are using LlamaIndex or LangGraph, you can find documentation on instrumenting them [here](https://langfuse.com/docs/integrations/llama-index/workflows) and [here](https://langfuse.com/docs/integrations/langchain/example-python-langgraph).

In [22]:
import os
import base64

from google.colab import userdata

os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com"
os.environ["LANGFUSE_PUBLIC_KEY"] = userdata.get('LANGFUSE_PUBLIC_KEY')
os.environ["LANGFUSE_SECRET_KEY"] = userdata.get('LANGFUSE_SECRET_KEY')

# Get your own keys from https://cloud.langfuse.com
LANGFUSE_AUTH = base64.b64encode(
    f"{userdata.get('LANGFUSE_PUBLIC_KEY')}:{userdata.get('LANGFUSE_SECRET_KEY')}".encode()
).decode()

os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = os.environ.get("LANGFUSE_HOST") + "/api/public/otel"
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"Authorization=Basic {LANGFUSE_AUTH}"

In [23]:
# Set your Hugging Face and other tokens/secrets as environment variable
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

In [24]:
from opentelemetry.sdk.trace import TracerProvider
from openinference.instrumentation.smolagents import SmolagentsInstrumentor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

# Create a TracerProvider for OpenTelemetry
trace_provider = TracerProvider()

# Add a SimpleSpanProcessor with the OTLPSpanExporter to send traces
trace_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter()))

# Set the global default tracer provider
from opentelemetry import trace
trace.set_tracer_provider(trace_provider)
tracer = trace.get_tracer(__name__)

# Instrument smolagents with the configured provider
SmolagentsInstrumentor().instrument(tracer_provider=trace_provider)


## Step 2: Test Your Instrumentation

Here is a simple CodeAgent from smolagents that calculates `1+1`. We run it to confirm that the instrumentation is working correctly. If everything is set up correctly, you will see logs/spans in your observability dashboard.

In [25]:
from smolagents import InferenceClientModel, CodeAgent

# Create a simple agent to test instrumentation
agent = CodeAgent(
    tools=[],
    model=InferenceClientModel()
)

agent.run("1+1=")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 1+1=                                                                                                            │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.11/dist-packages/smolagents/models.py:315: FutureWarning: Attribute last_input_token_count is deprecated and will be removed in version 1.20. Please use TokenUsage.input_tokens instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/smolagents/models.py:324: FutureWarning: Attribute last_output_token_count is deprecated and will be removed in version 1.20. Please use TokenUsage.output_tokens instead.
  warnings.warn(


─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  result = 1 + 1                                                                                                   
  final_answer(result)                                                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 2

[Step 1: Duration 0.74 seconds| Input tokens: 2,000 | Output tokens: 51]

2

Check your [Langfuse Traces Dashboard](https://cloud.langfuse.com/project/cman1g8qy00f8ad07m8kk34gk/traces) (or your chosen observability tool) to confirm that the spans and logs have been recorded.

Example screenshot from Langfuse:

![Example trace in Langfuse](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/bonus-unit2/first-example-trace.png)

_[Link to the trace](https://cloud.langfuse.com/project/cman1g8qy00f8ad07m8kk34gk/traces?peek=f98ae405d3b0bca24c4006a0ad42e6c9&timestamp=2025-06-15T07%3A25%3A33.462Z&observation=9c6ff1f34f65aac5)_

## Step 3: Observe and Evaluate a More Complex Agent

Now that you have confirmed your instrumentation works, let's try a more complex query so we can see how advanced metrics (token usage, latency, costs, etc.) are tracked.

In [26]:
%pip install duckduckgo-search

In [27]:
from smolagents import (CodeAgent, DuckDuckGoSearchTool, InferenceClientModel)

search_tool = DuckDuckGoSearchTool()
agent = CodeAgent(tools=[search_tool], model=InferenceClientModel())

agent.run("How many Rubik's Cubes could you fit inside the Notre Dame Cathedral?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ How many Rubik's Cubes could you fit inside the Notre Dame Cathedral?                                           │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.11/dist-packages/smolagents/models.py:315: FutureWarning: Attribute last_input_token_count is deprecated and will be removed in version 1.20. Please use TokenUsage.input_tokens instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/smolagents/models.py:324: FutureWarning: Attribute last_output_token_count is deprecated and will be removed in version 1.20. Please use TokenUsage.output_tokens instead.
  warnings.warn(


─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  cathedral_volume = web_search(query="Notre Dame Cathedral volume")                                               
  print(cathedral_volume)                                                                                          
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results

[List of largest church buildings - Wikipedia](https://en.wikipedia.org/wiki/List_of_largest_church_buildings)
Retrieved 3 December 2024. ^ "The People's Salvation Cathedral with its 14000 sqm and 120m height, is the tallest 
and largest Orthodox church in the world by volume". Marcegaglia Buildtech and SCBuildingEchipament (Italian 
company that provides the scaffolding of the cathedral / www.marcegagliabuildtech.com). 21 May 2020. Retrieved 30 
August 2020.

[Plans and Elevations - Notre-Dame de Paris](https://www.notredamedeparis.fr/en/understand/architecture/plans/)
Like most French cathedrals, Notre-Dame de Paris follows a Latin cross plan. Its main façade is oriented 
west-northwest, and its apse is oriented east-southeast. The cathedral can accommodate up to 9,000 people, 
including 1,500 in the galleries. Main Dimensions of the Cathedral Overall area: over 6,000 m² Length: 127 m Width:
48 m Height of the towers: 69 m Width of the façade: 43.5 m ...

[Notre Dame Cathedral - Data, Photos & Plans - 
WikiArquitectura](https://en.wikiarquitectura.com/building/Notre-Dame-Cathedral/)
Introduction Notre Dame de Paris is undoubtedly one of the symbols of the city. While not the largest cathedral in 
France, is one of his most remarkable Gothic art. Bishop Maurice de Sully began construction of the new cathedral 
from 1163 on the site occupied by a former church dedicated to St. Etienne. The construction coincided with a 
period of prosperity and increasing dominance of the city ...

[The Cathedral of Notre Dame in Paris, France, covers an area of 4,800 ...](https://brainly.com/question/51016375)
Thus, the volume of the interior of the Cathedral of Notre Dame in Paris is approximately 206,400 cubic meters. 
This impressive volume illustrates the grandeur of Gothic architecture and the cathedral's significant capacity. 
For example, if you wanted to find the volume of a different building, you would need the area of its base and its 
height.

[Notre Dame vs. Cologne Cathedral - Comparison of 
sizes](https://www.size-explorer.com/en/compare/buildings/notredame/colognecathedral/)
The cathedral is one of the most widely recognized symbols of the city of Paris and the French state. As the 
cathedral of the Archdiocese of Paris, Notre-Dame comprises the cathedra of the Archbishop of Paris (Michel 
Aupetit). In 1805, Notre-Dame was given the honorary status of a minor basilica.

[Gothic Grandeur: The Architectural Marvel of Notre 
Dame](https://www.re-thinkingthefuture.com/designing-for-typologies/a12951-gothic-grandeur-the-architectural-marvel
-of-notre-dame/)
A Latin cross is drawn in the cathedral's plan, as it is in most cathedrals. The transept creates the cross's 
horizontal arm. This volume is in line with the cathedral's measurements. In this area, religious ceremonies are 
held. The bottom part's walls were handled as external facades by Pierre de Montreuil in the thirteenth century, 
who strengthened the wall with a complex system of ...

[-Measurement plans (a) 1987 and (b) 2015 of the Notre-Dame 
cathedral.](https://www.researchgate.net/figure/Measurement-plans-a-1987-and-b-2015-of-the-Notre-Dame-cathedral_fig
1_308097722)
Notre-Dame de Paris is amongst the most well-known worship spaces in the world. Its large volume, in combination 
with a relatively bare stone construction and marble floor, leads to rather long ...

[Notre Dame Cathedral - New World Encyclopedia](https://www.newworldencyclopedia.org/entry/Notre_Dame_Cathedral)
Notre-Dame de Paris (French: Cathédrale Notre-Dame de Paris; meaning " Cathedral of Our Lady of Paris "), often 
referred to simply as Notre-Dame, [2] is a medieval Catholic cathedral on the Île de la Cité (an island in the 
River Seine), in the 4th arrondissement of Paris, France.

[ Acoustics of Notre-Dame cathedral de Paris](http://ica2016.org.ar/ica2016proceedings/ica2016/ICA2016-0269.pdf)
1 Introduction The Cathédrale Notre-Dame de Paris is amongst the most w

...

/usr/local/lib/python3.11/dist-packages/smolagents/models.py:315: FutureWarning: Attribute last_input_token_count is deprecated and will be removed in version 1.20. Please use TokenUsage.input_tokens instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/smolagents/models.py:324: FutureWarning: Attribute last_output_token_count is deprecated and will be removed in version 1.20. Please use TokenUsage.output_tokens instead.
  warnings.warn(


─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer(1114512968)                                                                                         
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 1114512968

[Step 3: Duration 4.60 seconds| Input tokens: 9,502 | Output tokens: 443]

1114512968

### Trace Structure

Most observability tools record a **trace** that contains **spans**, which represent each step of your agent’s logic. Here, the trace contains the overall agent run and sub-spans for:
- The tool calls (DuckDuckGoSearchTool)
- The LLM calls (InferenceClientModel)

You can inspect these to see precisely where time is spent, how many tokens are used, and so on:

![Trace tree in Langfuse](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/bonus-unit2/trace-tree.png)

_[Link to the trace](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/1ac33b89ffd5e75d4265b62900c348ed?timestamp=2025-03-07T13%3A45%3A09.149Z&display=preview)_

## Online Evaluation

In the previous section, we learned about the difference between online and offline evaluation. Now, we will see how to monitor your agent in production and evaluate it live.

### Common Metrics to Track in Production

1. **Costs** — The smolagents instrumentation captures token usage, which you can transform into approximate costs by assigning a price per token.
2. **Latency** — Observe the time it takes to complete each step, or the entire run.
3. **User Feedback** — Users can provide direct feedback (thumbs up/down) to help refine or correct the agent.
4. **LLM-as-a-Judge** — Use a separate LLM to evaluate your agent’s output in near real-time (e.g., checking for toxicity or correctness).

Below, we show examples of these metrics.

#### 1. Costs

Below is a screenshot showing usage for `Qwen2.5-Coder-32B-Instruct` calls. This is useful to see costly steps and optimize your agent.

![Costs](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/bonus-unit2/smolagents-costs.png)

_[Link to the trace](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/1ac33b89ffd5e75d4265b62900c348ed?timestamp=2025-03-07T13%3A45%3A09.149Z&display=preview)_

#### 2. Latency

We can also see how long it took to complete each step. In the example below, the entire conversation took 32 seconds, which you can break down by step. This helps you identify bottlenecks and optimize your agent.

![Latency](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/bonus-unit2/smolagents-latency.png)

_[Link to the trace](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/1ac33b89ffd5e75d4265b62900c348ed?timestamp=2025-03-07T13%3A45%3A09.149Z&display=preview)_

#### 3. Additional Attributes

You may also pass additional attributes—such as user IDs, session IDs, or tags—by setting them on the spans. For example, smolagents instrumentation uses OpenTelemetry to attach attributes like `langfuse.user.id` or custom tags.

In [28]:
from smolagents import (CodeAgent, DuckDuckGoSearchTool, InferenceClientModel)
from opentelemetry import trace

search_tool = DuckDuckGoSearchTool()
agent = CodeAgent(
    tools=[search_tool],
    model=InferenceClientModel()
)

with tracer.start_as_current_span("Smolagent-Trace") as span:
    span.set_attribute("langfuse.user.id", "smolagent-user-renwei")
    span.set_attribute("langfuse.session.id", "smolagent-session-20250615")
    span.set_attribute("langfuse.tags", ["city-question", "testing-agents"])

    agent.run("What is the capital of Germany?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What is the capital of Germany?                                                                                 │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.11/dist-packages/smolagents/models.py:315: FutureWarning: Attribute last_input_token_count is deprecated and will be removed in version 1.20. Please use TokenUsage.input_tokens instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/smolagents/models.py:324: FutureWarning: Attribute last_output_token_count is deprecated and will be removed in version 1.20. Please use TokenUsage.output_tokens instead.
  warnings.warn(


─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  capital_of_germany = web_search(query="capital of Germany")                                                      
  print(capital_of_germany)                                                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results

[Berlin - Wikipedia](https://en.wikipedia.org/wiki/Berlin)
Berlin (/ b ɜːr ˈ l ɪ n / bur-LIN; German: [bɛʁˈliːn] ⓘ) [9] is the capital and largest city of Germany, by both 
area and population. [10] With 3.7 million inhabitants, [5] it has the highest population within its city limits of
any city in the European Union.The city is also one of the states of Germany, being the third smallest state in the
country by area.

[Berlin | History, Map, Population, Attractions, & Facts | Britannica](https://www.britannica.com/place/Berlin)
Berlin is the capital and chief urban center of Germany. The city lies at the heart of the North German Plain, 
athwart an east-west commercial and geographic axis that helped make it the capital of the kingdom of Prussia and 
then, from 1871, of a unified Germany.

[What Is The Capital Of Germany? - 
WorldAtlas](https://www.worldatlas.com/articles/what-is-the-capital-of-germany.html)
Learn about the history, geography, and attractions of Berlin, the largest and most populous city in Germany. Find 
out how Berlin became the capital of Germany after the reunification of East and West Germany in 1990.

[What is the Capital of Germany? - Mappr](https://www.mappr.co/capital-cities/germany/)
Learn about Berlin, the capital of Germany, and its rich and turbulent history, diverse culture, and vibrant 
economy. Find out where Berlin is located, how it became the capital, and what attractions it offers.

[Berlin - Simple English Wikipedia, the free encyclopedia](https://simple.wikipedia.org/wiki/Berlin)
Berlin is the capital city of Germany and the largest city in the European Union. It has a rich history, a diverse 
culture, and many famous landmarks and attractions.

[What is the Capital of Germany? - Maps of 
World](https://www.mapsofworld.com/answers/world/what-is-the-capital-of-germany/)
After World War I, Weimar became the capital of the Weimar Republic, which was the first democratic government in 
Germany. However, it was a difficult time with problems like political instability and economic issues. Bonn. After
World War II, Germany split into two parts, and Bonn served as the capital of West Germany from 1949 to 1990. They 
...

[Capital of the Federal Republic of Germany](https://knowgermany.com/capital-of-the-federal-republic-of-germany/)
Learn about the history, landmarks, and culture of Berlin, the capital of Germany. Find out why Berlin was chosen 
as the capital of a reunified Germany and how it became a symbol of unity and a hub for art and innovation.

[What is the capital, language and currency of 
Germany?](https://traveltogermanyguide.com/what-is-the-capital-language-and-currency-of-germany/)
Learn about the capital, language and currency of Germany, a country with a rich history and culture. Find out how 
to explore Berlin, the capital city, and use the Euro, the official currency of the European Union.

[Capital of Germany - Wikipedia](https://en.wikipedia.org/wiki/Capital_of_Germany)
Prior to 1871, Germany was not a unified nation-state, and had no capital city.The medieval German Holy Roman 
Empire used to have Aachen as its preferred seat of government during Charlemagne's reign, and until 1531 it was 
the place where 31 Holy Roman Emperors were crowned Kings of the Germans. The coronation later moved to 
Frankfurt.However, after Charlemagne, none of the subsequent Holy ...

[Berlin - the city where anything is possible - Germany 
Travel](https://www.germany.travel/en/inspiring-germany/berlin.html)
Berlin is a vibrant and diverse city that combines historical landmarks, modern architecture, art, music and 
nightlife. Learn more about its attractions, museums, parks and events on the official tourism website.

Out: None

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.11/dist-packages/smolagents/models.py:315: FutureWarning: Attribute last_input_token_count is deprecated and will be removed in version 1.20. Please use TokenUsage.input_tokens instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/smolagents/models.py:324: FutureWarning: Attribute last_output_token_count is deprecated and will be removed in version 1.20. Please use TokenUsage.output_tokens instead.
  warnings.warn(


─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("Berlin")                                                                                           
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Berlin

[Step 4: Duration 3.77 seconds| Input tokens: 12,460 | Output tokens: 167]

![Enhancing agent runs with additional metrics](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/bonus-unit2/smolagents-attributes.png)

#### 4. User Feedback

If your agent is embedded into a user interface, you can record direct user feedback (like a thumbs-up/down in a chat UI). Below is an example using [Gradio](https://gradio.app/) to embed a chat with a simple feedback mechanism.

In the code snippet below, when a user sends a chat message, we capture the OpenTelemetry trace ID. If the user likes/dislikes the last answer, we attach a score to the trace.

In [29]:
import gradio as gr
from opentelemetry.trace import format_trace_id
from smolagents import (CodeAgent, InferenceClientModel)
from langfuse import Langfuse

langfuse = Langfuse()
model = InferenceClientModel()
agent = CodeAgent(tools=[], model=model, add_base_tools=True)

formatted_trace_id = None  # We'll store the current trace_id globally for demonstration

def respond(prompt, history):
    with trace.get_tracer(__name__).start_as_current_span("Smolagent-Trace") as span:
        output = agent.run(prompt)

        current_span = trace.get_current_span()
        span_context = current_span.get_span_context()
        trace_id = span_context.trace_id
        global formatted_trace_id
        formatted_trace_id = str(format_trace_id(trace_id))
        langfuse.trace(id=formatted_trace_id, input=prompt, output=output)

    history.append({"role": "assistant", "content": str(output)})
    return history

def handle_like(data: gr.LikeData):
    # For demonstration, we map user feedback to a 1 (like) or 0 (dislike)
    if data.liked:
        langfuse.score(
            value=1,
            name="user-feedback",
            trace_id=formatted_trace_id
        )
    else:
        langfuse.score(
            value=0,
            name="user-feedback",
            trace_id=formatted_trace_id
        )

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="Chat", type="messages")
    prompt_box = gr.Textbox(placeholder="Type your message...", label="Your message")

    # When the user presses 'Enter' on the prompt, we run 'respond'
    prompt_box.submit(
        fn=respond,
        inputs=[prompt_box, chatbot],
        outputs=chatbot
    )

    # When the user clicks a 'like' button on a message, we run 'handle_like'
    chatbot.like(handle_like, None, None)

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://de4e92357e2bb9b2ca.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


User feedback is then captured in your observability tool:

![User feedback is being captured in Langfuse](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/bonus-unit2/user-feedback-gradio.png)

#### 5. LLM-as-a-Judge

LLM-as-a-Judge is another way to automatically evaluate your agent's output. You can set up a separate LLM call to gauge the output’s correctness, toxicity, style, or any other criteria you care about.

**Workflow**:
1. You define an **Evaluation Template**, e.g., "Check if the text is toxic."
2. Each time your agent generates output, you pass that output to your "judge" LLM with the template.
3. The judge LLM responds with a rating or label that you log to your observability tool.

Example from Langfuse:

![LLM-as-a-Judge Evaluation Template](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/bonus-unit2/evaluator-template.png)
![LLM-as-a-Judge Evaluator](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/bonus-unit2/evaluator.png)

In [ ]:
# Example: Checking if the agent’s output is toxic or not.
from smolagents import (CodeAgent, DuckDuckGoSearchTool, InferenceClientModel)

search_tool = DuckDuckGoSearchTool()
agent = CodeAgent(tools=[search_tool], model=InferenceClientModel())

agent.run("Can eating carrots improve your vision?")

You can see that the answer of this example is judged as "not toxic".

![LLM-as-a-Judge Evaluation Score](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/bonus-unit2/llm-as-a-judge-score.png)

#### 6. Observability Metrics Overview

All of these metrics can be visualized together in dashboards. This enables you to quickly see how your agent performs across many sessions and helps you to track quality metrics over time.

![Observability metrics overview](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/bonus-unit2/langfuse-dashboard.png)

## Offline Evaluation

Online evaluation is essential for live feedback, but you also need **offline evaluation**—systematic checks before or during development. This helps maintain quality and reliability before rolling changes into production.

### Dataset Evaluation

In offline evaluation, you typically:
1. Have a benchmark dataset (with prompt and expected output pairs)
2. Run your agent on that dataset
3. Compare outputs to the expected results or use an additional scoring mechanism

Below, we demonstrate this approach with the [GSM8K dataset](https://huggingface.co/datasets/gsm8k), which contains math questions and solutions.

In [30]:
%pip install fsspec==2023.6.0

In [31]:
import fsspec
print(fsspec.__version__)

2023.6.0


In [32]:
import pandas as pd
from datasets import load_dataset

# Fetch GSM8K from Hugging Face
dataset = load_dataset("openai/gsm8k", 'main', split='train')
df = pd.DataFrame(dataset)
print("First few rows of GSM8K dataset:")
print(df.head())

First few rows of GSM8K dataset:
                                            question  \
0  Natalia sold clips to 48 of her friends in Apr...   
1  Weng earns $12 an hour for babysitting. Yester...   
2  Betty is saving money for a new wallet which c...   
3  Julie is reading a 120-page book. Yesterday, s...   
4  James writes a 3-page letter to 2 different fr...   

                                              answer  
0  Natalia sold 48/2 = <<48/2=24>>24 clips in May...  
1  Weng earns 12/60 = $<<12/60=0.2>>0.2 per minut...  
2  In the beginning, Betty has only 100 / 2 = $<<...  
3  Maila read 12 x 2 = <<12*2=24>>24 pages today....  
4  He writes each friend 3*2=<<3*2=6>>6 pages a w...  


Next, we create a dataset entity in Langfuse to track the runs. Then, we add each item from the dataset to the system. (If you’re not using Langfuse, you might simply store these in your own database or local file for analysis.)

In [35]:
from langfuse import Langfuse
langfuse = Langfuse()

langfuse_dataset_name = "gsm8k_dataset_huggingface"

# Create a dataset in Langfuse
langfuse.create_dataset(
    name=langfuse_dataset_name,
    description="GSM8K benchmark dataset uploaded from Huggingface",
    metadata={
        "date": "2025-03-10",
        "type": "benchmark"
    }
)

Dataset(id='cmbxdufhm04g0ad07vi4d79ge', name='gsm8k_dataset_huggingface', description='GSM8K benchmark dataset uploaded from Huggingface', metadata={'date': '2025-03-10', 'type': 'benchmark'}, project_id='cman1g8qy00f8ad07m8kk34gk', created_at=datetime.datetime(2025, 6, 15, 8, 7, 34, 379000, tzinfo=datetime.timezone.utc), updated_at=datetime.datetime(2025, 6, 15, 8, 7, 34, 379000, tzinfo=datetime.timezone.utc))

In [36]:
for idx, row in df.iterrows():
    langfuse.create_dataset_item(
        dataset_name=langfuse_dataset_name,
        input={"text": row["question"]},
        expected_output={"text": row["answer"]},
        metadata={"source_index": idx}
    )
    if idx >= 9: # Upload only the first 10 items for demonstration
        break

![Dataset items in Langfuse](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/bonus-unit2/example-dataset.png)

#### Running the Agent on the Dataset

We define a helper function `run_smolagent()` that:
1. Starts an OpenTelemetry span
2. Runs our agent on the prompt
3. Records the trace ID in Langfuse

Then, we loop over each dataset item, run the agent, and link the trace to the dataset item. We can also attach a quick evaluation score if desired.

In [37]:
from opentelemetry.trace import format_trace_id
from smolagents import (CodeAgent, InferenceClientModel, LiteLLMModel)

# Example: using InferenceClientModel or LiteLLMModel to access openai, anthropic, gemini, etc. models:
model = InferenceClientModel()

agent = CodeAgent(
    tools=[],
    model=model,
    add_base_tools=True
)

def run_smolagent(question):
    with tracer.start_as_current_span("Smolagent-Trace") as span:
        span.set_attribute("langfuse.tag", "dataset-run")
        output = agent.run(question)

        current_span = trace.get_current_span()
        span_context = current_span.get_span_context()
        trace_id = span_context.trace_id
        formatted_trace_id = format_trace_id(trace_id)

        langfuse_trace = langfuse.trace(
            id=formatted_trace_id,
            input=question,
            output=output
        )
    return langfuse_trace, output

In [20]:
print(dir(langfuse))

['__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_create_observation_id', '_create_remote_parent_span', '_create_span_with_parent_context', '_environment', '_fetch_prompt_and_update_cache', '_format_otel_span_id', '_format_otel_trace_id', '_get_bounded_max_retries', '_get_current_otel_span', '_get_otel_span_id', '_get_otel_trace_id', '_get_project_id', '_host', '_is_valid_span_id', '_is_valid_trace_id', '_mask', '_otel_tracer', '_project_id', '_resources', '_start_as_current_otel_span_with_processed_media', '_tracing_enabled', '_url_encode', 'api', 'async_api', 'auth_check', 'create_dataset', 'create_dataset_item', 'create_event', 'create_prompt', 'create_scor

In [38]:
dataset = langfuse.get_dataset(langfuse_dataset_name)

# Run our agent against each dataset item (limited to first 10 above)
for item in dataset.items:
    langfuse_trace, output = run_smolagent(item.input["text"])

    # Link the trace to the dataset item for analysis
    item.link(
        langfuse_trace,
        run_name="smolagent-notebook-run-01",
        run_metadata={ "model": model.model_id }
    )

    # Optionally, store a quick evaluation score for demonstration
    langfuse_trace.score(
        name="<example_eval>",
        value=1,
        comment="This is a comment"
    )

# Flush data to ensure all telemetry is sent
langfuse.flush()

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Tina makes $18.00 an hour.  If she works more than 8 hours per shift, she is eligible for overtime, which is    │
│ paid by your hourly wage + 1/2 your hourly wage.  If she works 10 hours every day for 5 days, how much money    │
│ does she make?                                                                                                  │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in generating model output:
402 Client Error: Payment Required for url: 
https://router.huggingface.co/hf-inference/models/Qwen/Qwen2.5-Coder-32B-Instruct/v1/chat/completions (Request ID: 
Root=1-684e7f60-496df61d71df97d349257f1a;b52686d6-8f8f-42fa-8f73-4f06b2f0bb00)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly 
included credits.

[Step 1: Duration 0.86 seconds]

AgentGenerationError: Error in generating model output:
402 Client Error: Payment Required for url: https://router.huggingface.co/hf-inference/models/Qwen/Qwen2.5-Coder-32B-Instruct/v1/chat/completions (Request ID: Root=1-684e7f60-496df61d71df97d349257f1a;b52686d6-8f8f-42fa-8f73-4f06b2f0bb00)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.

You can repeat this process with different:
- Models (OpenAI GPT, local LLM, etc.)
- Tools (search vs. no search)
- Prompts (different system messages)

Then compare them side-by-side in your observability tool:

![Dataset run overview](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/bonus-unit2/dataset_runs.png)
![Dataset run comparison](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/bonus-unit2/dataset-run-comparison.png)


## Final Thoughts

In this notebook, we covered how to:
1. **Set up Observability** using smolagents + OpenTelemetry exporters
2. **Check Instrumentation** by running a simple agent
3. **Capture Detailed Metrics** (cost, latency, etc.) through an observability tools
4. **Collect User Feedback** via a Gradio interface
5. **Use LLM-as-a-Judge** to automatically evaluate outputs
6. **Perform Offline Evaluation** with a benchmark dataset

🤗 Happy coding!